In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from typing import Literal
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage

In [ ]:
from langgraph.graph import END, START, StateGraph, MessagesState
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode

In [ ]:
@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["munich"]:
        return "It's 15 degrees Celsius and cloudy."
    else:
        return "It's 32 degrees Celsius and sunny."

In [ ]:
tools = [get_weather]
model = ChatOpenAI(model="gpt-4o-mini").bind_tools(tools)

In [ ]:
def call_model(state: MessagesState):
    messages = state["messages"]
    response = model.invoke(messages)
    return {"messages": [response]}


def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

In [ ]:
workflow = StateGraph(MessagesState)
tool_node = ToolNode(tools)

In [ ]:
workflow = StateGraph(MessagesState)

workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
)
workflow.add_edge("tools", "agent")

graph = workflow.compile(checkpointer=MemorySaver())

In [ ]:
graph.invoke(
    {"messages": [HumanMessage(content="How is the weather in munich?")]},
    config={"configurable": {"thread_id": 1}},
)

In [ ]:
graph.invoke(
    {
        "messages": [
            HumanMessage(content="What would you recommend to do in that city then?")
        ]
    },
    config={"configurable": {"thread_id": 1}},
)

### Getting production ready - async and streaming

In [ ]:
model = ChatOpenAI(model="gpt-4o-mini", streaming=True).bind_tools(tools)

In [ ]:
async def call_model(state: MessagesState):
    messages = state["messages"]
    response = await model.ainvoke(messages)
    return {"messages": [response]}

In [ ]:
workflow = StateGraph(MessagesState)
tool_node = ToolNode(tools)

workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {"tools": "tools", END: END},
)
workflow.add_edge("tools", "agent")

graph = workflow.compile(checkpointer=MemorySaver())

In [ ]:
inputs = {"messages": [HumanMessage(content="How is the weather in Munich?")]}
config = {"configurable": {"thread_id": 2}}

In [ ]:
await graph.ainvoke(input=inputs, config=config)

In [ ]:
inputs = {"messages": [HumanMessage(content="How is the weather in Munich?")]}
async for output in graph.astream(inputs, stream_mode="updates", config=config):
    # stream_mode="updates" yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value["messages"][-1].pretty_print())

In [ ]:
from langchain_core.messages import AIMessageChunk, HumanMessage

inputs = [HumanMessage(content="How is the weather in Munich?")]
gathered = None

async for msg, metadata in graph.astream(
    {"messages": inputs}, stream_mode="messages", config=config
):
    if msg.content and not isinstance(msg, HumanMessage):
        # Print each token as it streams in
        print(msg.content, end="|", flush=True)

    # Handle the AI message chunks for proper assembly
    if isinstance(msg, AIMessageChunk):
        if gathered is None:
            gathered = msg
        else:
            gathered = gathered + msg

In [ ]:
print(gathered.content)